In [1]:
import numpy as np
import pandas as pd
import pyddm
import os
import models

In [10]:
def get_model_measures(model, condition):
    sol = model.solve(condition)
    mean_rt_error = np.sum(sol.pdf_err()*model.t_domain())*model.dt / sol.prob_error()
    return condition["tta_0"], condition["d_0"], condition["a_values"], condition["a_duration"],\
           sol.prob_correct(), sol.mean_decision_time(), mean_rt_error

def get_model_rt_distr(model, condition, kind="cdf"):
    sol = model.solve(condition)
    return pd.DataFrame({"tta_0": condition["tta_0"],
                         "d_0": condition["d_0"],
                         "a_values": condition["a_values"],
                         "a_duration": condition["a_duration"],
                         "t": model.t_domain(),
                         "rt_corr_distr": (sol.cdf_corr() if kind=="cdf" else sol.pdf_corr())/sol.prob_correct(),
                         "rt_error_distr": (sol.cdf_err() if kind=="cdf" else sol.pdf_err())/sol.prob_error()})

def initialize_model(model_no, param_set):
    overlay = models.OverlayNonDecisionGaussian(ndt_location=param_set.ndt_location, ndt_scale=param_set.ndt_scale)
    bound = models.BoundCollapsingTta(b_0=param_set.b_0, k=param_set.k, tta_crit=param_set.tta_crit)
    drift = models.DriftAccelerationDependent(alpha=param_set.alpha, beta_d=param_set.beta_d, beta_a=param_set.beta_a, theta=param_set.theta)

    model = pyddm.Model(name="Model %i" % model_no, drift=drift, bound=bound, overlay=overlay,
                      noise=pyddm.NoiseConstant(noise=1), T_dur=models.ModelAccelerationDependent.T_dur)

    return model

def simulate_model(model_no, param_set, conditions, ret="measures"):
    """
    Set ret to "measures" or "rt_cdf" or "rt_pdf" for saving p_turn and mean RT or RT CDF or RT PDF
    """
    model = initialize_model(model_no, param_set)

    if ret=="measures":
        sim_result = pd.DataFrame([get_model_measures(model, condition) for condition in conditions],
                                  columns=["tta_0", "d_0", "a_values", "a_duration", "is_gap_accepted", "RT_go", "RT_stay"])
    else:
        sim_result = pd.concat([get_model_rt_distr(model, condition, kind=ret[-3:]) for condition in conditions])
    sim_result["subj_id"] = param_set.subj_id
    return sim_result

In [22]:
def save_sim_results(model_no, file_name, parameters=None, conditions=None, cross_validation=False, ret="measures", prefix=""):
    model_name = "model_acceleration_dependent"

    if cross_validation:
        model_name += "_cross_validation"

    path = os.path.join("modeling/fit_results_excluded_nan_rt", model_name)

    if parameters is None:
        parameters = pd.read_csv(os.path.join(path, file_name))

    sim_results = [simulate_model(model_no, param_set, conditions, ret=ret)
                   for idx, param_set in parameters.iterrows()]

    sim_results = pd.concat(sim_results)
    sim_results.to_csv(os.path.join(path, (prefix+file_name).replace("parameters_fitted", "sim_" + ret)), index=False)

In [12]:
conditions = [{"tta_0": tta_0, "d_0": d_0, "a_values": a_values, "a_duration": a_duration}
              for tta_0 in np.linspace(4, 6, 9)
              for d_0 in np.linspace(70, 90, 3)
              for a_values in [(0., 0., 0., 0.),
                               (0., 4, 4, 0.),
                               (0., 4, -4, 0.),
                               (0., -4, 4, 0.),
                               (0., -4, -4, 0.)]
              for a_duration in [1.0]]

# Model parameters

In [13]:
path = "modeling/fit_results_excluded_nan_rt/model_acceleration_dependent_cross_validation"
parameters = pd.read_csv(os.path.join(path, "subj_all_parameters_fitted.csv"))

In [14]:
parameters

,subj_id,loss,alpha,beta_d,beta_a,theta,b_0,k,tta_crit,ndt_location,ndt_scale
0,all,5544.163499,0.44495,0.143334,0.008694,14.288385,2.634552,0.01133,8.009755,0.019708,0.312318


# Saving model-predicted measures

In [15]:
save_sim_results(model_no=3, file_name="subj_all_parameters_fitted.csv", conditions=conditions, cross_validation=True, ret="measures")

# Testing the effects of $\beta_a$

In [23]:
path = "modeling/fit_results_excluded_nan_rt/model_acceleration_dependent_cross_validation"
parameters = pd.read_csv(os.path.join(path, "subj_all_parameters_fitted.csv"))
parameters["beta_a"] = 0

save_sim_results(model_no=3, file_name="subj_all_parameters_fitted.csv", parameters=parameters, conditions=conditions, cross_validation=True, ret="measures", prefix="beta_a_0_")

In [25]:
path = "modeling/fit_results_excluded_nan_rt/model_acceleration_dependent_cross_validation"
parameters = pd.read_csv(os.path.join(path, "subj_all_parameters_fitted.csv"))
parameters["beta_a"] = 1

save_sim_results(model_no=3, file_name="subj_all_parameters_fitted.csv", parameters=parameters, conditions=conditions, cross_validation=True, ret="measures", prefix="beta_a_1_")

# Predicting the effect of other nudges

In [10]:
a_durations = np.linspace(0.1, 3.0, 11)
a_magnitudes = np.linspace(0.0, 5.0, 11)

conditions = [{"tta_0": 5, "d_0": 90, "a_values": (0.0, -a_magnitude, a_magnitude, 0.0), "a_duration": a_duration}
              for a_duration in a_durations
              for a_magnitude in a_magnitudes]

In [11]:
save_sim_results(model_no=3, file_name="subj_all_parameters_fitted.csv", conditions=conditions, cross_validation=True,
                 ret="measures", prefix="prediction_")